In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

os.chdir('C:/Users/icear/OneDrive/바탕 화면/새 폴더/porto-seguro-safe-driver-prediction')

sns.set_palette("pastel")
sns.set_style('whitegrid')

In [2]:
train = pd.read_csv('train_new.csv')
test = pd.read_csv('test_new.csv')

In [3]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, plot_importance
import lightgbm as lgbm
from sklearn.model_selection import cross_val_score

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

- 지니계수가 평가항목이니 지니계수 정의
- LightGBM에 맞게 지니계수 정의

In [4]:
X = train.drop(['target'], axis = 1)
features = X.columns
X = X.values
y = train['target'].values

nrounds = 400
kfold = 5


params = {'metric': 'auc', 
          'learning_rate' : 0.01, 
          'max_depth':10, 
          'max_bin':10,  
          'objective': 'binary', 
          'feature_fraction': 0.8,
          'bagging_fraction':0.9,
          'bagging_freq':10,  
          'min_data': 500}

skf = StratifiedKFold(n_splits=kfold, random_state=1, shuffle=True)
for i, (train_index, test_index) in enumerate(skf.split(X,y)):
    print(' lgb kfold: {} of {} : '.format(i+1,kfold))
    X_train, X_eval = X[train_index], X[test_index]
    y_train, y_eval = y[train_index], y[test_index]
    
    lgb_model = lgbm.train(params, 
                          lgbm.Dataset(X_train, label=y_train), 
                          nrounds, 
                          lgbm.Dataset(X_eval, label=y_eval), 
                          verbose_eval=100, 
                          feval=gini_lgb, 
                          early_stopping_rounds=100)

 lgb kfold: 1 of 5 : 
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Info] Number of positive: 415814, number of negative: 415814
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.165220 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 454
[LightGBM] [Info] Number of data points in the train set: 831628, number of used features: 187
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.617649	valid_0's gini: 0.235295
[200]	valid_0's auc: 0.628279	valid_0's gini: 

- LightGBM을 적용시키고 지니계수로 평가
- StratifiedKFold를 사용해 fold마다 label의 분포를 같게 해줌